In [1]:
import pandas as pd
import random 
import os

pd.set_option('display.max_rows', 50)


from functions import sharpe_ratio_calculation, generate_rand_portfolios, select_top_five, join_stocks_crypto, test_for_silhouette_score, run_clustering_model, label_balance

c:\Users\Kirill\Documents\Projects\thesis\venv312\Lib\site-packages\tslearn\bases\bases.py:15: UserWarning: h5py not installed, hdf5 features will not be supported.
Install h5py to use hdf5 features: http://docs.h5py.org/
  warn(h5py_msg)


In [2]:
df_all_stocks = pd.read_csv('stocks_data_filled.csv',index_col='Date')
cryptos_df = pd.read_csv('cryptos_data.csv', index_col='Date')

joined_df = join_stocks_crypto(cryptos_df, df_all_stocks, mode = 'stocks_left') #mode - either do left with crypto and fill NA for stocks or do left on stocks and leave out some dates for cryptos
joined_df.index = pd.to_datetime(joined_df.index)

joined_df_weekly = joined_df.resample('W').last() #try aggregating on a weekly level
joined_df_3days = joined_df.resample('3D').last()# aggregating on a twice per week basis to arrive at the sweet spot of that 250 (1 year) timeseries length

In [3]:
#Random Portfolios generation
tickers = list(df_all_stocks.columns)

random.seed(42)
random_portfolios = generate_rand_portfolios(n_reps=1000, n_stocks=15, tickers=tickers)

#Select top five sharpe ratio portfolios from a portfolio
sharpe_ratio = sharpe_ratio_calculation(df_all_stocks, rf_rate_annual = 0.02)
top_five_dict = select_top_five(random_portfolios, metric=sharpe_ratio)

In [4]:
#RUN THE CLUSTERING WITH DIFFERENT SET UPS TO GET THE SILHOUETTE SCORES FOR COMPARISON

# n_clusters_list = [4,5,6,7]
# linkage_list=['single', 'average', 'complete']
# #window_sizes = [3,7,10,14,21,30,60]
# window_sizes = [21,30,60]


# def run_clustering_evaluation(df, window_sizes, method, moving_average=True, return_mode='arithmetic', df_input_name='DFWASNOTSPECIFIED'):

#     for w_size in window_sizes:

#         #return_mode = 'arithmetic'
#         #n_init = 3
#         #center = True
#         if moving_average:
#             df = df.rolling(window=w_size, center=True).mean()

#             smoothing = 'moving_average'
#         else:
#             smoothing = 'no_smoothing'

#         if len(df) < 150 and w_size > 30:
#             continue
#         silhouette_df = test_for_silhouette_score(df, n_clusters_list, method=method, return_mode=return_mode, n_init=3, linkage_list=linkage_list)

#         silhouette_df['return_mode'] = return_mode
#         silhouette_df['n_init'] = 3
#         silhouette_df['smoothing'] = smoothing
#         silhouette_df['window_size/span'] = w_size

#         silhouette_df.to_csv(f'silhouette_dfs/{method}_{smoothing}_{return_mode}_windowsize-{w_size}_{df_input_name}.csv')




# df_input_name = input('Put in the name of the df mode you are running for: ')
# for return_mode in ['arithmetic', 'geometric']:
#     run_clustering_evaluation(joined_df_3days, window_sizes, method='kmeans', moving_average=True, return_mode=return_mode, df_input_name=df_input_name)

In [5]:
#RUN FOR LABEL BALANCE

# method_loop = 'ahc'
# return_mode_loop = 'geometric'
# n_clus_loop = 7

# df_with_label_balance = pd.DataFrame()
# for df_dict in [{'weekly': joined_df_weekly}, {'3day': joined_df_3days}, {'full': joined_df}]:
#     for linkage in ['single', 'complete', 'average']:
#         for w in [3,7,10,14,21,30]:
#             output = label_balance(df_dict, w, method_loop, return_mode_loop, n_clus=n_clus_loop, linkage=linkage)
#             df_with_label_balance = pd.concat([df_with_label_balance, output])

# df_with_label_balance.reset_index(inplace=True)

#df_with_label_balance.to_csv(f'balance_datasets/balance_test_results_{method_loop}_{return_mode_loop}_{n_clus_loop}clusters.csv')

In [6]:
##############TEST###################
# from functions import test_clustering_metrics
# n_clusters_list = [4,5,6,7,9,12]
# df_with_label_balance = pd.DataFrame()
# for window in [3,7,10,14,30,60]:
#     for df_dict in [{'weekly': joined_df_weekly}, {'3day': joined_df_3days}, {'full': joined_df}]:
#         output = test_clustering_metrics(df_dict, n_clusters_list, method='kmeans', linkage_list=None,#['average', 'complete', 'single'], 
#                                return_mode='arithmetic', window=window, n_init=3)
#         df_with_label_balance = pd.concat([df_with_label_balance, output])

# df_with_label_balance.to_csv('new_balance_silhouette/kmeans.csv')

In [7]:
def rejoin_dfs(folder_path, extract_cols_from_filename=False):
    '''
    extract_cols_from_filename is the option to handle legacy thing with the silhouette score tables when part of the information was stored in the filename
    '''
    dfs = pd.DataFrame()
    for filename in os.listdir(folder_path):
        file_path = os.path.join(folder_path, filename)  
        df = pd.read_csv(file_path, index_col=False)
        if extract_cols_from_filename:
            df['filename'] = filename
        dfs = pd.concat([dfs, df])
    for col in ['index', 'Unnamed: 0']:
        try:
            dfs = dfs.drop(columns=[col])
        except:
            pass
    if extract_cols_from_filename:
        dfs['df_mode'] = dfs['filename'].apply(lambda x: x.split('-')[1].split('_')[1].replace('.csv', ''))
        dfs['return_mode'] = dfs['filename'].apply(lambda x: x.split('_')[3])
        dfs['linkage'] = dfs['linkage'].fillna('not_applicable')
    
    dfs = dfs.drop_duplicates()

    return dfs

In [8]:
folder_path_sil = 'silhouette_dfs/'
folder_path_balance = 'balance_datasets/'

silhouette_results = rejoin_dfs(folder_path_sil, extract_cols_from_filename=True).drop(columns=['smoothing', 'n_init', 'inertia', 'filename'])
balance_results = rejoin_dfs(folder_path_balance).drop(columns=['filename'])


silhouette_results_over_15percent = silhouette_results[silhouette_results['silhouette_score'] >= 0.15]

In [9]:
silhouette_results

,clusters,silhouette_score,method,linkage,return_mode,window_size/span,df_mode
0,4,0.450849,ahc,single,arithmetic,10,3day
1,5,0.366116,ahc,single,arithmetic,10,3day
2,6,0.217847,ahc,single,arithmetic,10,3day
3,7,0.186290,ahc,single,arithmetic,10,3day
4,4,0.436554,ahc,average,arithmetic,10,3day
...,...,...,...,...,...,...,...
3,7,0.175246,kshape,not_applicable,geometric,7,full
0,4,0.122094,kshape,not_applicable,geometric,7,weekly
1,5,0.109347,kshape,not_applicable,geometric,7,weekly
2,6,0.106075,kshape,not_applicable,geometric,7,weekly


In [10]:
balance_results = balance_results.drop_duplicates(subset=['return_mode', 'clusters', 'window_size', 'method', 'linkage', 'df_mode'])

In [11]:
df_all_results = pd.merge(silhouette_results, balance_results, how='left', 
                          left_on=['clusters', 'linkage', 'window_size/span', 'df_mode', 'method', 'return_mode'],
                          right_on=['clusters', 'linkage', 'window_size', 'df_mode', 'method', 'return_mode'])


from sklearn.preprocessing import MinMaxScaler
df_all_results[['silhouette_norm']] = MinMaxScaler().fit_transform(df_all_results[['silhouette_score']])
df_all_results['delta_norm'] = 1 - MinMaxScaler().fit_transform(df_all_results[['min_max_delta']])
df_all_results['total_score'] = (df_all_results['silhouette_norm'] + df_all_results['delta_norm']) / 2

best_configs = df_all_results.loc[df_all_results.groupby('method')['total_score'].idxmax()]

In [12]:
# df_rest = pd.DataFrame()
# for index, r in x.iterrows():
#     df_dict = {'weekly':{'weekly': joined_df_weekly}, '3day': {'3day': joined_df_3days}, 'full': {'full': joined_df}}[r['df_mode']]


#     output = label_balance(df_dict=df_dict, window=r['window_size/span'], method=r['method'], return_mode=r['return_mode'], n_clus=r['clusters'], linkage=r['linkage'])
#     df_rest = pd.concat([df_rest, output])

# df_rest.to_csv(f'balance_datasets/rest.csv')

In [13]:
#what are the lowest silhouette scores?
#silhouette_results[silhouette_results['silhouette_score'] <= 0.1]#.sort_values(by='silhouette_score')

#what is the mean for silhouette score per method?
df_all_results.groupby(by='method')['silhouette_score'].mean()

method
ahc       0.203292
kmeans    0.126730
kshape    0.108682
Name: silhouette_score, dtype: float64

In [25]:
df_all_results_geometric.to_csv('full_table_silhouette_balance_geometric.csv')

In [15]:
df_all_results_geometric = df_all_results[df_all_results['return_mode'] == 'geometric']
df_all_results_arithmetic = df_all_results[df_all_results['return_mode'] == 'arithmetic']

df_all_results_arithmetic.groupby(by='method')['silhouette_score'].mean()
df_all_results_geometric.groupby(by='method')['silhouette_score'].mean()

method
ahc       0.197987
kmeans    0.136128
kshape    0.081889
Name: silhouette_score, dtype: float64

In [21]:
df_all_results_filtered = df_all_results_geometric[(df_all_results_geometric['silhouette_score'] > 0.1) & (df_all_results_geometric['df_mode'] != 'weekly')].reset_index() 

In [22]:
df_all_results_filtered

,index,clusters,silhouette_score,method,linkage,return_mode,window_size/span,df_mode,window_size,min_per_cluster,max_per_cluster,min_max_delta,silhouette_norm,delta_norm,total_score
0,240,4,0.381144,ahc,single,geometric,10,3day,10.0,0.0045,0.9685,0.9640,0.755263,0.018937,0.387100
1,241,5,0.331475,ahc,single,geometric,10,3day,10.0,0.0045,0.9640,0.9595,0.703531,0.023672,0.363601
2,242,6,0.331575,ahc,single,geometric,10,3day,10.0,0.0045,0.9550,0.9505,0.703634,0.033140,0.368387
3,243,7,0.232784,ahc,single,geometric,10,3day,10.0,0.0045,0.9505,0.9459,0.600740,0.037980,0.319360
4,244,4,0.277851,ahc,average,geometric,10,3day,10.0,0.0045,0.9550,0.9505,0.647679,0.033140,0.340410
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
197,791,7,0.147433,kshape,not_applicable,geometric,7,3day,7.0,0.0946,0.2342,0.1396,0.511845,0.886270,0.699057
198,792,4,0.129811,kshape,not_applicable,geometric,7,full,7.0,0.1351,0.4685,0.3333,0.493490,0.682483,0.587987
199,793,5,0.146321,kshape,not_applicable,geometric,7,full,7.0,0.1261,0.3784,0.2523,0.510687,0.767701,0.639194
200,794,6,0.172403,kshape,not_applicable,geometric,7,full,7.0,0.1081,0.2117,0.1036,0.537852,0.924145,0.730999


In [23]:
best_configs = df_all_results_filtered.loc[df_all_results_filtered.groupby('method')['total_score'].idxmax()]

In [24]:
best_configs

,index,clusters,silhouette_score,method,linkage,return_mode,window_size/span,df_mode,window_size,min_per_cluster,max_per_cluster,min_max_delta,silhouette_norm,delta_norm,total_score
109,431,7,0.428222,ahc,complete,geometric,60,3day,60.0,0.0090,0.2928,0.2838,0.804295,0.734561,0.769428
158,623,7,0.344045,kmeans,not_applicable,geometric,60,3day,60.0,0.0360,0.2703,0.2342,0.716622,0.786744,0.751683
200,794,6,0.172403,kshape,not_applicable,geometric,7,full,7.0,0.1081,0.2117,0.1036,0.537852,0.924145,0.730999


In [19]:
###CRYPTOS DISTRIBUTION IN CLUSTERS:
# from collections import Counter
# _, tickers_with_labels, _, _ = run_clustering_model(joined_df, n_clus=3, model_name='kmeans', linkage='single', return_mode='arithmetic', n_init=3)

# cryptos_list = list(cryptos_df.columns) 
# crypto_clusters = {ticker: tickers_with_labels[ticker] for ticker in cryptos_list if ticker in tickers_with_labels}

# # Count how many cryptos are in each cluster
# distribution = Counter(crypto_clusters.values())